In [50]:
#### To start we will import all of our needed libraries and dive into our datasets.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.impute import KNNImputer

from py_files.feature_engineering import filter_column_strings, knn_impute, cat_label_pct


In [51]:
df = pd.read_csv('data/post_eda_data.csv')
df.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)

/tmp/ipykernel_447730/2834474839.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/post_eda_data.csv')


In [52]:
df.head()

,id,status_group,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,NaN,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,NaN,Unicef,263.0,UNICEF,38.486161,-11.155298,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,NaN,Action In A,NaN,Artisan,31.130847,-1.825359,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


## To Do:

#### String Clean up
   
#### Construction year to age
#### KNNImpute
    - population
    - gps_height
    - amount_tsh¶
    
#### Bin population

#### Mask features based on value count



In [54]:
# String Clean up 

columns_ = [
    'funder',
    'installer',
    'scheme_name',
    'subvillage',
    'ward',
    'basin',
    'region',
    'lga',
    'ward',
    'scheme_management',
    'scheme_name',
    'extraction_type',
    'extraction_type_group',
    'extraction_type_class',
    'management',
    'management_group',
    'payment',
    'payment_type',
    'water_quality',
    'quality_group',
    'quantity',
    'quantity_group',
    'source',
    'source_type',
    'source_class',
    'waterpoint_type',
    'waterpoint_type_group'
]

for col_ in columns_:
    df[col_] = df[col_].str.replace(' ', '') \
        .str.lower() \
        .str.replace('(','') \
        .str.replace(')', '') \
        .str.replace('/', '') \
        .str.replace('_', '')
    
    filter_column_strings(90, df, col_, fuzz.token_sort_ratio, drop=True)

#### Mask low value count features

In [61]:
scheme_mgmt_idx = np.array(np.where(df['scheme_management'] == 'none')).tolist()[0]

df.loc[scheme_mgmt_idx, 'scheme_management'] = 'other'

masked_cols = ['funder', 'installer', 'subvillage', 'ward', 'scheme_name']

for col in masked_cols:
    mask_ = df[col].map(df[col].value_counts()) == 1
    df[col] = df[col].mask(mask_, 'other')

#### Construction Year to Age

In [67]:
df[df.construction_year != 0.0]['construction_year'].median()

2000.0

In [69]:
df.construction_year = df.construction_year.replace(0.0, 2000)

In [71]:
df['construction_year'] = 2022 - df['construction_year']
df = df.rename(columns={'construction_year': 'well_age'})

#### Mask low value count features

In [72]:
df.to_csv('data/cleaned_data.csv')

In [73]:
df = pd.read_csv('data/cleaned_data.csv')

/tmp/ipykernel_447730/3976309765.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_data.csv')


In [74]:
test_df = df[~df.status_group.notna()].copy()
test_df.drop('status_group', axis=1, inplace=True)
df = df[df.status_group.notna()].copy()

#### KNN Imputation

In [ ]:
knn_impute(df, test_df, 'population', [0,1], 66)
knn_impute(df, test_df, 'gps_height', 0, 66)
knn_impute(df, test_df, 'amount_tsh', 0, 5)

In [47]:
test_df.to_csv('data/test_set_imputed.txt')
df.to_csv('data/df_imputed.txt')

df = pd.read_csv('data/df_imputed.txt', index_col=[0])
test_df = pd.read_csv('data/test_set_imputed.txt', index_col=[0])

In [49]:
df.funder.value_counts()

governmentoftanzania    9084
unknown                 3639
dagida                  3116
hasawa                  2205
nrwssp                  1387
                        ... 
kwik                       2
zaben                      2
snv-swash                  2
africanbarrickgold         2
padri                      2
Name: funder, Length: 867, dtype: int64

#### Feature engineer percentage based on functionality

In [41]:
df[['region_code', 'district_code']] = df[['region_code', 'district_code']].astype('object')

In [42]:
columns = df.drop('status_group', axis=1).select_dtypes('object').columns.tolist()
for col_ in columns:
    if col_ != 'status_group':
        df, test_df = cat_label_pct(df, test_df, col_)
        df.drop(col_, axis=1, inplace=True)
        test_df.drop(col_, axis=1, inplace=True)

In [43]:
test_df.fillna(0.0, inplace=True)
df.fillna(0.0, inplace=True)

In [44]:
# Verifying ID's are identical in the test dataset and nan values
og_test_df = pd.read_csv('data/Test_set.csv')
pd.DataFrame(og_test_df['id'] == test_df['id'])['id'].value_counts()

True    14850
Name: id, dtype: int64

In [45]:
test_df.to_csv('data/ml_testing.txt')
df.to_csv('data/ml_training.txt')